In [ ]:
with open ("requirements.txt", "r") as file: 
    
    dependencies = [
    line.strip() for line in file.readlines()
    if '==' in line and not line.strip().startswith('#')
]

print ("Dependencies found: ", dependencies)

In [ ]:
import requests

def get_latest_version(package):
    try:
        url = f"https://pypi.org/pypi/{package}/json"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data['info']['version']
    except (requests.RequestException, KeyError):
        return("Unknown Package")


In [ ]:
from packaging.version import Version, InvalidVersion

def is_safe_to_update(current, latest):
    try: 
        current_ver = Version(current)
        latest_ver = Version(latest)
    except InvalidVersion:
        return False

    return (current_ver.major == latest_ver.major) and (current_ver.minor <= latest_ver.minor)


In [ ]:
from IPython.display import Markdown, display

results = []

for dep in dependencies:
    try:
        package, current_version = dep.split('==')
    except ValueError:
        print(f"Skipping invalid entry: {dep}")
        continue

    latest_version = get_latest_version(package)
    safe = "✅ Yes" if latest_version != "Unknown" and is_safe_to_update(current_version, latest_version) else "⚠️ No"
    results.append((package, current_version, latest_version, safe))

def generate_markdown_table(results):
    header = "| Package | Current Version | Latest Version | Safe to Update? |"
    separator = "| --- | --- | --- | --- |"
    rows = "\n".join([f"| {package} | {current_version} | {latest_version} | {safe} |" for package, current_version, latest_version, safe in results])
    return f"{header}\n{separator}\n{rows}"

markdown_table = generate_markdown_table(results)

display(Markdown(markdown_table))
